##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Generate Midi Sound with an RNN

<h2>Source tutorial at</h2>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/audio/music_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
</table>

## Setup

Importing the necessary Python modules

In [2]:
# !sudo apt install -y fluidsynth
# !pip install --upgrade pyfluidsynth
# !pip install pretty_midi

In [3]:
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Optional

In [4]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

## Load the Classical Music MIDI dataset

In [5]:
data_dir = pathlib.Path('data/Classical-Music-MIDI')

In [6]:
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

Number of files: 295


## Process a MIDI file

This section demonstrates loading and processing a MIDI file using Python's `pretty_midi` library. We will explore various aspects of the MIDI file, including displaying audio, identifying instruments, and extracting relevant information.

In [7]:
sample_file = filenames[1]
print(sample_file)

data\Classical-Music-MIDI\albeniz\alb_esp2.mid


In [8]:
pm = pretty_midi.PrettyMIDI(sample_file)

In [9]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [10]:
display_audio(pm)

In [11]:
print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

Number of instruments: 2
Instrument name: Acoustic Grand Piano


## Extract notes

This section demonstrates how to extract notes from a MIDI file and convert them into a structured format suitable for model processing. Defined functions to convert MIDI files into a DataFrame of notes, as well as to convert notes back into MIDI format.

In [12]:
for i, note in enumerate(instrument.notes[:10]):
  note_name = pretty_midi.note_number_to_name(note.pitch)
  duration = note.end - note.start

In [13]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  # Sort the notes by start time
  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [14]:
raw_notes = midi_to_notes(sample_file)
raw_notes.head()

,pitch,start,end,step,duration
0,57,0.718850,0.952059,0.000000,0.233209
1,62,0.952059,1.953045,0.233209,1.000987
2,66,0.952059,1.953045,0.000000,1.000987
3,69,0.952059,1.953045,0.000000,1.000987
4,57,2.671895,2.905104,1.719836,0.233209


In [15]:
def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str,
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

## Create the training dataset

This section prepares the training dataset for training a model on musical sequences. Extracts notes from multiple MIDI files, concatenate them and create sequences suitable for training. Create a TensorFlow Dataset from the sequences, which is shuffled, batched, and preprocessed to be ready for model training.

In [16]:
num_files = 5
all_notes = []
for f in filenames[:num_files]:
  notes = midi_to_notes(f)
  all_notes.append(notes)

all_notes = pd.concat(all_notes)
all_notes

,pitch,start,end,step,duration
0,81,0.321096,0.642192,0.000000,0.321096
1,88,0.642192,2.574745,0.321096,1.932553
2,86,2.574745,2.641412,1.932553,0.066667
3,88,2.641412,2.708079,0.066667,0.066667
4,86,2.708079,2.971262,0.066667,0.263184
...,...,...,...,...,...
509,71,117.032502,118.244624,0.000000,1.212122
510,87,118.244624,120.668868,1.212122,2.424244
511,99,118.244624,120.668868,0.000000,2.424244
512,94,118.244624,120.668868,0.000000,2.424244


In [17]:
n_notes = len(all_notes)

Next, create a `tf.data.Dataset` from the parsed notes.

In [18]:
key_order = ['pitch', 'step', 'duration']
train_notes = np.stack([all_notes[key] for key in key_order], axis=1)

In [19]:
notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)
notes_ds.element_spec

TensorSpec(shape=(3,), dtype=tf.float64, name=None)

In [20]:
def create_sequences(
    dataset: tf.data.Dataset,
    seq_length: int,
    vocab_size = 128,
) -> tf.data.Dataset:
  """Returns TF Dataset of sequence and label examples."""
  seq_length = seq_length+1

  # Take 1 extra for the labels
  windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True)

  # `flat_map` flattens the" dataset of datasets" into a dataset of tensors
  flatten = lambda x: x.batch(seq_length, drop_remainder=True)
  sequences = windows.flat_map(flatten)

  # Normalize note pitch
  def scale_pitch(x):
    x = x/[vocab_size,1.0,1.0]
    return x

  # Split the labels
  def split_labels(sequences):
    inputs = sequences[:-1]
    labels_dense = sequences[-1]
    labels = {key:labels_dense[i] for i,key in enumerate(key_order)}

    return scale_pitch(inputs), labels

  return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)

In [21]:
seq_length = 25
vocab_size = 128
seq_ds = create_sequences(notes_ds, seq_length, vocab_size)
seq_ds.element_spec

(TensorSpec(shape=(25, 3), dtype=tf.float64, name=None),
 {'pitch': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'step': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'duration': TensorSpec(shape=(), dtype=tf.float64, name=None)})

In [22]:
batch_size = 64
buffer_size = n_notes - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

In [23]:
train_ds.element_spec

(TensorSpec(shape=(64, 25, 3), dtype=tf.float64, name=None),
 {'pitch': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'step': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'duration': TensorSpec(shape=(64,), dtype=tf.float64, name=None)})

## Create and train the model

This section defines and train a neural network model for generating music sequences. The model architecture consists of an LSTM layer followed by dense layers for predicting pitch, step, and duration of musical notes. Also define custom loss functions and optimizer settings suitable for training.



In [24]:
def mse_with_positive_pressure(y_true: tf.Tensor, y_pred: tf.Tensor):
  mse = (y_true - y_pred) ** 2
  positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
  return tf.reduce_mean(mse + positive_pressure)

In [25]:
input_shape = (seq_length, 3)
learning_rate = 0.01

inputs = tf.keras.Input(input_shape)
x = tf.keras.layers.LSTM(128)(inputs)

outputs = {
  'pitch': tf.keras.layers.Dense(128, name='pitch')(x),
  'step': tf.keras.layers.Dense(1, name='step')(x),
  'duration': tf.keras.layers.Dense(1, name='duration')(x),
}

model = tf.keras.Model(inputs, outputs)

loss = {
      'pitch': tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True),
      'step': mse_with_positive_pressure,
      'duration': mse_with_positive_pressure,
}

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

model.compile(loss=loss, optimizer=optimizer)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25, 3)]              0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 128)                  67584     ['input_1[0][0]']             
                                                                                                  
 duration (Dense)            (None, 1)                    129       ['lstm[0][0]']                
                                                                                                  
 pitch (Dense)               (None, 128)                  16512     ['lstm[0][0]']                
                                                                                             

In [26]:
losses = model.evaluate(train_ds, return_dict=True)
losses



45/45 [==============================] - 1s 6ms/step - loss: 6.5336 - duration_loss: 0.9121 - pitch_loss: 4.8536 - step_loss: 0.7678


{'loss': 6.533580303192139,
 'duration_loss': 0.9121443033218384,
 'pitch_loss': 4.853622913360596,
 'step_loss': 0.7678123712539673}

In [27]:
model.compile(
    loss=loss,
    loss_weights={
        'pitch': 0.05,
        'step': 1.0,
        'duration':1.0,
    },
    optimizer=optimizer,
)

In [28]:
model.evaluate(train_ds, return_dict=True)

45/45 [==============================] - 1s 5ms/step - loss: 1.9226 - duration_loss: 0.9121 - pitch_loss: 4.8536 - step_loss: 0.7678


{'loss': 1.9226378202438354,
 'duration_loss': 0.9121443033218384,
 'pitch_loss': 4.853622913360596,
 'step_loss': 0.7678123712539673}

### Train model

In [29]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        verbose=1,
        restore_best_weights=True),
]

In [40]:
%%time
epochs = 50

history = model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
)

Epoch 1/50
45/45 [==============================] - 0s 10ms/step - loss: 0.2886 - duration_loss: 0.0844 - pitch_loss: 3.0828 - step_loss: 0.0501
Epoch 2/50
45/45 [==============================] - 0s 9ms/step - loss: 0.2923 - duration_loss: 0.0853 - pitch_loss: 3.0796 - step_loss: 0.0530
Epoch 3/50
45/45 [==============================] - 0s 10ms/step - loss: 0.2910 - duration_loss: 0.0848 - pitch_loss: 3.0763 - step_loss: 0.0524
Epoch 4/50
45/45 [==============================] - 0s 10ms/step - loss: 0.2878 - duration_loss: 0.0841 - pitch_loss: 3.0740 - step_loss: 0.0500
Epoch 5/50
45/45 [==============================] - 0s 9ms/step - loss: 0.2886 - duration_loss: 0.0841 - pitch_loss: 3.0699 - step_loss: 0.0510
Epoch 6/50
45/45 [==============================] - 0s 10ms/step - loss: 0.2896 - duration_loss: 0.0845 - pitch_loss: 3.0665 - step_loss: 0.0518
Epoch 7/50
45/45 [==============================] - 0s 9ms/step - loss: 0.2867 - duration_loss: 0.0823 - pitch_loss: 3.0605 - step_l

## Save the trained model

This section saves the trained music generation model and its weights for future use. The model architecture and trained weights are stored in files named 'trained_midi_gen' and 'trained_midi_gen_weights', respectively. This lets us easily reload the model for inference or further training in subsequent sessions.

In [44]:
model.save('trained_midi_gen')
model.save_weights('trained_midi_gen_weights')

INFO:tensorflow:Assets written to: trained_midi_gen\assets


INFO:tensorflow:Assets written to: trained_midi_gen\assets
